# Data Preparation

In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import os
import sys
module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

import psycopg2
import pandas as pd
import numpy as np
#import dbpass
pd.set_option('max_colwidth',0)

In [6]:
DBNAME = "opportunity_youth"
conn = psycopg2.connect(dbname=DBNAME)
#conn = psycopg2.connect(dbname=DBNAME, user="postgres") #<<<THIS WAS NOT WORKING FOR ME SO I COMMENTED OUT --Angelica 
#conn = psycopg2.connect(dbname=DBNAME, user="postgres", password=dbpass.postgrepass())

In [7]:
query = """
SELECT cow, esr, schl education_attained, fschp enrollment_status, agep age
FROM pums_2017
where puma between '11610' and '11615'
or puma between '11604' and '11605'
"""
df = pd.read_sql(query, conn)
df['employed'] = 1

df.loc[(df.cow == '3') | (df.esr == '9'),'employed'] = 0
df.loc[(df.cow == np.nan) & (df.esr == np.nan), 'employed'] = 0
df.drop(columns = ['cow','esr'], inplace = True)
df['education_attained'].fillna(df['education_attained'].mode()[0],inplace = True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45511 entries, 0 to 45510
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   education_attained  45511 non-null  object 
 1   enrollment_status   45511 non-null  object 
 2   age                 45511 non-null  float64
 3   employed            45511 non-null  int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 1.4+ MB


In [8]:
df.head()

,education_attained,enrollment_status,age,employed
0,05,0,40.0,1
1,06,0,11.0,1
2,04,0,9.0,1
3,11,0,48.0,1
4,11,0,48.0,1


puma = region (11613 - 11615 for S. King County)

agep = age (16-24)

cow = employment (9 = unemployed)

fschp = school enrollment (1 = yes, 0 = no)

esr = employment recode (3 = unemployed)

fesrp employment recode (0 - not employed)

In [46]:
#added population weights (dict code 'pwgtp') column and re-named it
query_wg = """
SELECT cow, esr, schl education_attained, fschp enrollment_status, agep age, pwgtp population_weight
FROM pums_2017
where puma between '11610' and '11615'
or puma between '11604' and '11605'
"""
df_wg = pd.read_sql(query_wg, conn)
df_wg['employed'] = 1

df_wg.loc[(df_wg.cow == '3') | (df_wg.esr == '9'),'employed'] = 0
df_wg.loc[(df_wg.cow == np.nan) & (df_wg.esr == np.nan), 'employed'] = 0
df_wg.drop(columns = ['cow','esr'], inplace = True)
df_wg['education_attained'].fillna(df_wg['education_attained'].mode()[0],inplace = True)

df_wg.info()
df_wg.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45511 entries, 0 to 45510
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   education_attained  45511 non-null  object 
 1   enrollment_status   45511 non-null  object 
 2   age                 45511 non-null  float64
 3   population_weight   45511 non-null  float64
 4   employed            45511 non-null  int64  
dtypes: float64(2), int64(1), object(2)
memory usage: 1.7+ MB


,education_attained,enrollment_status,age,population_weight,employed
0,05,0,40.0,90.0,1
1,06,0,11.0,78.0,1
2,04,0,9.0,60.0,1
3,11,0,48.0,109.0,1
4,11,0,48.0,108.0,1


***Total Opportunity Youth --> df_total_OY***
Pulling the total number of opportunity youth as defined by: "not in school, not working" from 2016 report. ##NEED TO CITE PROPERLY IF WE USE###

In [96]:
df_total_OY = df_wg[(df_wg['employed'] == 0) & (df_wg['age'].between(16,24) & (df_wg['enrollment_status'] == '0'))]
df_total_OY


,education_attained,enrollment_status,age,population_weight,employed
667,19,0,23.0,4.0,0
951,21,0,22.0,15.0,0
1846,16,0,18.0,14.0,0
2397,14,0,20.0,5.0,0
2484,16,0,19.0,11.0,0
...,...,...,...,...,...
43801,21,0,24.0,63.0,0
43936,16,0,21.0,27.0,0
43972,19,0,24.0,67.0,0
44004,13,0,17.0,16.0,0


***By Age Opportunity Youth --> df_age_OY***

In [128]:
#Having trouble with this, not familiar with binning so not sure I am doing how Josh envisioned. I tried several things but was not being successful.

bins = [16, 19, 22, np.inf]
names = ['16-18', '19-21', '22-24']

df_total_OY['age_range'] = pd.cut(df_total_OY['age'], bins, labels=names)
df_total_OY

<ipython-input-128-0b81df25fe66>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_total_OY['age_range'] = pd.cut(df_total_OY['age'], bins, labels=names)


,education_attained,enrollment_status,age,population_weight,employed,AgeRange,age_range
667,19,0,23.0,4.0,0,22-24,22-24
951,21,0,22.0,15.0,0,19-21,19-21
1846,16,0,18.0,14.0,0,16-18,16-18
2397,14,0,20.0,5.0,0,19-21,19-21
2484,16,0,19.0,11.0,0,16-18,16-18
...,...,...,...,...,...,...,...
43801,21,0,24.0,63.0,0,22-24,22-24
43936,16,0,21.0,27.0,0,19-21,19-21
43972,19,0,24.0,67.0,0,22-24,22-24
44004,13,0,17.0,16.0,0,16-18,16-18


***Total Working Without a Diploma ( and also out of school) --> df_total_working***



In [87]:
df_total_working = df_wg[(df_wg.education_attained.isin(["15", "19"])) 
        & (df_wg['employed'] == 1) & (df_wg['age'].between(16,24)) 
        & (df_wg['enrollment_status'] == '0')]
df_total_working

#what if yes high school diploma (15) and no college diploma (19)?

,education_attained,enrollment_status,age,population_weight,employed
67,19,0,24.0,41.0,1
69,19,0,22.0,47.0,1
72,15,0,18.0,24.0,1
146,19,0,19.0,91.0,1
170,19,0,20.0,16.0,1
...,...,...,...,...,...
45439,19,0,20.0,14.0,1
45440,19,0,20.0,7.0,1
45464,19,0,19.0,17.0,1
45471,19,0,20.0,17.0,1


***By Age Working Without Diploma --> df_age_working***

In [ ]:
#insert code once binning has been resolved above

***Total Not Oportunity Youth --> df_total_NOY***

In [99]:
df_total_NOY = df_wg[(df_wg.education_attained.isin(['15', '19'])) 
        & (df_wg['employed'] == 1) & (df_wg['age'].between(16,24)) 
        & (df_wg.enrollment_status.isin(['0', '1']))]
df_total_NOY

,education_attained,enrollment_status,age,population_weight,employed
67,19,0,24.0,41.0,1
69,19,0,22.0,47.0,1
72,15,0,18.0,24.0,1
146,19,0,19.0,91.0,1
170,19,0,20.0,16.0,1
...,...,...,...,...,...
45439,19,0,20.0,14.0,1
45440,19,0,20.0,7.0,1
45464,19,0,19.0,17.0,1
45471,19,0,20.0,17.0,1


***By Age Not Opportunity Youth--> df_age_NOY***

In [ ]:
#insert code once binning has been resolved above

***Opportunity Youth No Diploma --> df_OY_ND***


In [102]:
#Using data dictionary: '15' = 12th grade - no diploma  and  '19'=1 or more years of college credit, no degree

df_OY_ND = df_total_OY[(df_total_OY.education_attained.isin(['15', '19']))]
df_OY_ND

,education_attained,enrollment_status,age,population_weight,employed
667,19,0,23.0,4.0,0
3151,19,0,20.0,6.0,0
4307,19,0,20.0,17.0,0
7633,15,0,20.0,26.0,0
9415,19,0,22.0,13.0,0
11868,19,0,23.0,40.0,0
13106,19,0,24.0,3.0,0
14731,19,0,18.0,13.0,0
16946,19,0,21.0,17.0,0
17733,19,0,20.0,16.0,0


***Opportunity Youth High School Diploma or GED --> df_OY_HS***

In [103]:
##Using data dictionary: '16' = Regular high school diploma ,  '17' = GED or alternative credential
df_OY_HS = df_total_OY[(df_total_OY.education_attained.isin(['16', '17']))]
df_OY_HS

,education_attained,enrollment_status,age,population_weight,employed
1846,16,0,18.0,14.0,0
2484,16,0,19.0,11.0,0
4089,16,0,21.0,5.0,0
7767,16,0,18.0,16.0,0
8099,16,0,18.0,85.0,0
8374,17,0,24.0,32.0,0
10231,16,0,21.0,24.0,0
18032,16,0,19.0,12.0,0
18186,16,0,23.0,4.0,0
18469,16,0,24.0,18.0,0


***Opportunity Youth Some College, No Degree --> df_OY_SC***

In [127]:
#Using data dictionary: '18' =  Some college, but less than 1 year ,  '19' = 1 or more years of college credit, no degree
df_OY_SC = df_total_OY[(df_total_OY.education_attained.isin(['18', '19']))]
df_OY_SC

,education_attained,enrollment_status,age,population_weight,employed,AgeRange,age_range
667,19,0,23.0,4.0,0,22-24,22-24
3151,19,0,20.0,6.0,0,19-21,19-21
4307,19,0,20.0,17.0,0,19-21,19-21
9415,19,0,22.0,13.0,0,19-21,19-21
11868,19,0,23.0,40.0,0,22-24,22-24
13106,19,0,24.0,3.0,0,22-24,22-24
13230,18,0,22.0,17.0,0,19-21,19-21
14731,19,0,18.0,13.0,0,16-18,16-18
16946,19,0,21.0,17.0,0,19-21,19-21
17733,19,0,20.0,16.0,0,19-21,19-21


***Opportunity Youth Degree(Associate or Higher) --> df_OY_YD***

In [105]:
#Using data dictionary: 20-24: Associates, masters, doctorate, etc
df_OY_YD = df_total_OY[(df_total_OY['education_attained'] >= '20')]
df_OY_YD

,education_attained,enrollment_status,age,population_weight,employed
951,21,0,22.0,15.0,0
4438,20,0,21.0,16.0,0
7288,20,0,24.0,15.0,0
8978,21,0,22.0,17.0,0
9553,21,0,22.0,17.0,0
10770,21,0,24.0,23.0,0
13763,20,0,22.0,28.0,0
14384,21,0,24.0,12.0,0
14486,20,0,22.0,17.0,0
20459,21,0,23.0,35.0,0


In [ ]:
conn.close()